In [1]:
!pip install scikit-surprise

  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-win_amd64.whl size=734853 sha256=10e533c9577795da56221a55d374e1ae07755fbe43feae0af34b667832cbffee
  Stored in directory: c:\users\dlwog\appdata\local\pip\cache\wheels\20\91\57\2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data =Dataset.load_builtin('ml-100k')
trainset,testset = train_test_split(data, test_size=.25,random_state=0)


Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\dlwog/.surprise_data/ml-100k


In [6]:
algo = SVD()
algo.fit(trainset)

In [7]:
predictions = algo.test(testset)
print('prediction type:', type(predictions), 'size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5004101792334255, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.6386389920837305, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.7929043766975963, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.555108416122742, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.3336311949408675, details={'was_impossible': False})]

In [8]:
[(pred.uid, pred.iid,pred.est)for pred in predictions[:3] ]

[('120', '282', 3.5004101792334255),
 ('882', '291', 3.6386389920837305),
 ('535', '507', 3.7929043766975963)]

In [9]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력
uid = str(196)
iid = str(302)
pred = algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.34   {'was_impossible': False}


In [10]:
accuracy.rmse(predictions)

RMSE: 0.9506


0.9506107566153716

In [13]:
import pandas as pd
ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv')
ratings.to_csv('./dataset/ml-latest-small/ratings_noh.csv', index=False, header=False)

In [14]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',',rating_scale=(0.5,5))
data = Dataset.load_from_file('./dataset/ml-latest-small/ratings_noh.csv',reader=reader)

trainset, testset = train_test_split(data, test_size=.25, random_state=0)
algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

[과제] ratings.csv 데이터로 SVD 클래스를 이용하여 아래와 같이 추천 알고리즘을 빌딩하고 평가하세요
- 교차 검증 및 하이퍼파라미터 튜닝 : measures(RMSE, MAE), cv =5, param = n_epoch(20,40,60), n_factors(50,100,200

In [15]:
from surprise.model_selection import cross_validate
# 판다스 데이터 프레임에서 surprise데이터 세트로 데이터 로딩

ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

algo = SVD(random_state=0)
cross_validate(algo,data,measures=['RMSE','MAE'],cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8786  0.8703  0.8762  0.8668  0.8763  0.8737  0.0044  
MAE (testset)     0.6742  0.6706  0.6730  0.6658  0.6735  0.6714  0.0030  
Fit time          8.78    8.66    8.44    8.55    8.60    8.61    0.12    
Test time         0.26    0.26    0.39    0.26    0.29    0.29    0.05    


{'test_rmse': array([0.87859165, 0.87033225, 0.876186  , 0.86684771, 0.87629707]),
 'test_mae': array([0.67415732, 0.67062882, 0.67298103, 0.66582624, 0.67354835]),
 'fit_time': (8.783970832824707,
  8.659059286117554,
  8.435168266296387,
  8.552103519439697,
  8.597078084945679),
 'test_time': (0.25785207748413086,
  0.2608330249786377,
  0.38977718353271484,
  0.2608511447906494,
  0.28583741188049316)}

In [17]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태 지정
param_grid = {'n_epochs':[20,40,60],'n_factors':[50,100,200] }

# CV를 5개 폴트 세트로 지정, 성능은 rmse, mse 수행
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'],cv=5)
gs.fit(data)

# 최고 RMS Evaluation점수와 그떄의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8716115252047109
{'n_epochs': 20, 'n_factors': 50}


[과제] 개인화 영화 추천 시스템을 작성(잠재요인 협업 필터링 방식)
- 데이터셋: ratings_noh.csv,movies.csv

In [19]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
#DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='./dataset/ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [20]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [24]:
# 영화에 대한 상세 속성 종보 데이터프레임으로 로딩
movies = pd.read_csv('./dataset/ml-latest-small/movies.csv')

# user id = 9사용자의 movieId데이터를 추출해 movieId=42데이터 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
    
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [25]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid,iid,verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [27]:
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    
    #모든 영화의 movieId 를 리스트로 생성
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후에 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies),
         '전체 영화 수:', len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings,movies,9)
    

평점 매긴 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [28]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [(id, title, rating) for id, title, rating in \
                      zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
    print(top_movie[1],":", top_movie[2])

평점 매긴 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
#### Top-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
